In [22]:
import pandas as pd
import sklearn.tree as sktree
import scipy as sp
import os
import datatable as dt
path = "./데이터/"
filename = os.listdir(path)

In [23]:
target = pd.read_csv(f"{path}침수지역2022.csv")

In [24]:
print("한번이라도 침수가 일어난 지역수(행정동) :", target["ADM_NM"].nunique())

한번이라도 침수가 일어난 지역수(행정동) : 277


#### 침수가 일어난 지역(행정동)

In [25]:
target["ADM_NM"].unique()

array(['우이동', '영등포동', '송중동', '묵1동', '대림1동', '신길6동', '신대방1동', '신길5동',
       '방학2동', '한강로동', '방학1동', '도림동', '신길3동', '회현동', '번1동', '상계5동', '장충동',
       '필동', '서초3동', '다산동', '수유3동', '동화동', '약수동', '상계3·4동', '길동', '방배2동',
       '장위3동', '보광동', '인수동', '남영동', '쌍문3동', '역삼1동', '도곡1동', '반포2동',
       '방배본동', '성수1가1동', '고덕1동', '장위1동', '성수2가1동', '상일동', '삼양동', '장안1동',
       '장안2동', '회기동', '진관동', '용신동', '전농1동', '제기동', '이문1동', '답십리1동',
       '면목본동', '면목3·8동', '면목2동', '면목7동', '방배1동', '상봉1동', '방배3동', '면목4동',
       '대림2동', '구로3동', '조원동', '서초1동', '망우본동', '망우3동', '신내1동', '대림3동',
       '돈암1동', '삼선동', '남가좌2동', '동선동', '월곡1동', '정릉3동', '공덕동', '종암동',
       '개포1동', '장위2동', '반포1동', '충현동', '석관동', '방배4동', '영등포본동', '정릉2동',
       '천호3동', '서초4동', '문래동', '성내2동', '창5동', '미아동', '송천동', '수유2동', '사당1동',
       '번2동', '수유1동', '도봉1동', '쌍문2동', '도봉2동', '쌍문1동', '창1동', '창2동',
       '쌍문4동', '창3동', '방학3동', '월계2동', '공릉1동', '상계1동', '청파동', '상계2동',
       '상계10동', '중화1동', '중계본동', '월계3동', '공항동', '반포4동', '불광2동', '응암3동',
       '성

In [26]:
경사도 = pd.read_excel(path + "/경사도.xlsx")
도로율 = pd.read_csv(path + "/" + "도로현황(도로율)_20231015181102.csv")
배수등급양호비율 = pd.read_csv(path + "/" + "배수등급양호비율.csv")
하천까지의거리 = pd.read_csv(path + "/" + "하천까지의_거리.csv")
인구밀도 = pd.read_csv(path + "/" + "서울시_행정동별_인구밀도_2023.csv")
빗물펌프장까지의거리 = pd.read_excel(path + "/" + "거리계산.xlsx")
월평균소비 = pd.read_csv(path + "/"+ "서울시 상권분석서비스(소득소비-행정동)_20231101_서울열린데이터광장.csv", encoding = 'euc-kr')
관공서합계 = pd.read_excel(path + "/" + "인프라_행정동별_관공서합계(경찰서,병원,소방서).xlsx")
행정동별30년이상건물개수 = pd.read_csv(path + "/" + "행정동별30년이상건물개수.csv")

#### 사회인프라

#### 강우량

In [28]:
path3 = path + "서울시_강우량_정보_2022년/"

In [29]:
rain_dict = {}
for i in range(1, 13):
    rain_dict[i] = pd.read_csv(path3 + f"서울시_강우량_정보_2022년{i}월.csv", encoding="euc-kr")

In [30]:
rain_data = pd.concat([rain_dict[i][rain_dict[i]["10분우량"] != 0] for i in range(1, 13)])
rain_agg = rain_data.groupby(["구청명"])["10분우량"].agg(["sum", "count"])
rain_agg.rename(
    columns=dict(zip(["sum", "count"], ["누적강수량", "10분당 측정(횟수)"])), inplace=True
)
rain_agg.index.rename(None, inplace=True)

In [31]:
rain_agg["추정시우량(연간)"] = rain_agg["누적강수량"] / (rain_agg["10분당 측정(횟수)"] / 6)

In [32]:
rain_agg.index.rename("시군구", inplace=True)
rain_agg.reset_index(inplace=True)

In [33]:
구면적 = pd.DataFrame(인구밀도.groupby(["구"])["면적 (k㎡)"].sum()).reset_index()
구면적.rename(columns={"구": "시군구"}, inplace=True)

In [34]:
rain_agg = pd.merge(rain_agg, 구면적)
rain_agg["추정면적당시우량(연간)"] = rain_agg["추정시우량(연간)"] / rain_agg["면적 (k㎡)"]

In [35]:
rain_agg

,시군구,누적강수량,10분당 측정(횟수),추정시우량(연간),면적 (k㎡),추정면적당시우량(연간)
0,강남구,5187.0,3025,10.288264,39.51,0.260396
1,강동구,3426.5,2300,8.938696,24.59,0.363509
2,강북구,1803.5,1382,7.829957,23.60,0.331778
3,강서구,2690.5,2053,7.863127,41.44,0.189747
4,관악구,3461.0,2176,9.543199,29.57,0.322732
5,광진구,1887.0,1353,8.368071,17.06,0.490508
6,구로구,3246.5,1962,9.928135,20.12,0.493446
7,금천구,3270.0,2091,9.383070,13.02,0.720666
8,노원구,3290.5,2312,8.539360,35.44,0.240953
9,도봉구,1932.0,1435,8.078049,20.65,0.391189


### 데이터 병합

행정동 단위
> 경사도, 행정공시지가, 배수등급양호비율, 하천까지의 거리, 제방수문암거

In [36]:
import PublicDataReader as pdr

행정동코드 = pdr.code_hdong()
import numpy as np

행정동 = 행정동코드[행정동코드["시도명"] == "서울특별시"][["시군구명", "읍면동명"]]
행정동.index = 행정동.index.astype("int")
행정동["시군구명"] = 행정동["시군구명"].str.strip()
행정동["읍면동명"] = 행정동["읍면동명"].str.strip()
행정동 = 행정동[행정동["시군구명"] != ""][행정동["읍면동명"] != ""]

출처: 행정기관(행정동) 및 관할구역(법정동) 변경내역(2023.10.30. 시행)


C:\Users\yis82\AppData\Local\Temp\ipykernel_13636\1195263744.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  행정동 = 행정동[행정동["시군구명"] != ""][행정동["읍면동명"] != ""]


In [37]:
per_경사도 = pd.DataFrame(경사도.groupby(["ADM_NM"])["경사도"].mean())
per_경사도.reset_index(inplace=True)

In [38]:
per_경사도 = pd.merge(경사도[["GU_NAME", "ADM_NM"]], per_경사도, how="left")
per_경사도["경사도(%)"] = per_경사도["경사도"] * 100
per_경사도.drop_duplicates(["ADM_NM"], inplace=True)

In [39]:
X = pd.read_excel(path + "/" + "행정동단위통합데이터.xlsx")

In [40]:
X.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'GU_NAME', 'ADM_NM', '배수등급양호비율', '제방',
       '수문', '평균경사도(%)', '하천까지의거리(중심)', '인구밀도(명/km²)', '1분위소득가구비율(%)',
       '2분위소득가구비율(%)', '3분위소득가구비율(%)', '4분위소득가구비율(%)', '5분위소득가구비율(%)',
       '빗물펌프장까지의거리(m)', '토출량(㎥/분)', '펌프(합산)', '노후건물수', '월_평균_소득_금액', '지출_총금액',
       '경찰', '소방서', '병원', '침수횟수'],
      dtype='object')

In [42]:
행정동별_녹지면적 = pd.read_csv("./데이터/행정동별 녹지면적.csv")
행정동_면적 = pd.read_csv("./데이터/행정구역(동별)_20231115212741_열린데이터광장_2022.csv")
행정동별_녹지면적["행정동별_녹지면적(km²)"] = 행정동별_녹지면적["GREEN_AREA"] / 1000

In [43]:
도로 = pd.read_csv("./데이터/행정동별_도로.csv")
건물 = pd.read_csv("./데이터/건물.csv")

In [44]:
도로길이 = pd.read_csv("./데이터/도로 길이.csv")
도로길이

,ADM_NM,LENGTH
0,사직동,23837.242371
1,삼청동,15490.784853
2,부암동,29312.630201
3,평창동,53320.458893
4,무악동,5646.249268
...,...,...
419,둔촌1동,4415.158442
420,둔촌2동,21017.148085
421,암사1동,22690.302000
422,천호2동,36252.632690


In [45]:
도로폭 = 도로.groupby("ADM_NM")[['ROAD_BT']].mean()
도로폭

,ROAD_BT
ADM_NM,
가락1동,21.476190
가락2동,10.601695
가락본동,10.466667
가리봉동,4.968531
가산동,12.356784
...,...
효창동,5.691176
후암동,5.042169
휘경1동,4.811331


In [46]:
# Selecting specific columns for each dataframe
건물 = 건물[["ADM_NM", "SHAPE_AREA"]]
도로 = pd.merge(도로폭, 도로길이, on="ADM_NM")

# Calculating ROAD_AREA by multiplying ROAD_BT and ROAD_LT
도로["ROAD_AREA"] = 도로["ROAD_BT"] / 1000 * 도로["LENGTH"] / 1000

In [47]:
도로.loc[도로['ADM_NM'] =='가산동']

,ADM_NM,ROAD_BT,LENGTH,ROAD_AREA
4,가산동,12.356784,32994.692374,0.407708


In [48]:
# Grouping by ADM_NM and summing the values for both 도로 and 건물
건물 = 건물.groupby("ADM_NM").sum()

# Merging the two dataframes on "ADM_NM"
도로_건물 = pd.merge(도로, 건물, on="ADM_NM")

# Adding a new column "도로_건물_면적" by summing ROAD_AREA and SHAPE_AREA
도로_건물["도로_건물_면적"] = 도로_건물["ROAD_AREA"] + 도로_건물["SHAPE_AREA"]/1000000

In [49]:
도로_건물['도로_건물_면적_km²'] = 도로_건물['도로_건물_면적']

In [50]:
도로_건물

,ADM_NM,ROAD_BT,LENGTH,ROAD_AREA,SHAPE_AREA,도로_건물_면적,도로_건물_면적_km²
0,가락1동,21.476190,6242.503821,0.134065,341973.83,0.476039,0.476039
1,가락2동,10.601695,18692.242008,0.198169,219508.10,0.417678,0.417678
2,가락본동,10.466667,21562.538246,0.225688,253786.39,0.479474,0.479474
3,가리봉동,4.968531,17142.757591,0.085174,168897.44,0.254072,0.254072
4,가산동,12.356784,32994.692374,0.407708,817992.38,1.225701,1.225701
...,...,...,...,...,...,...,...
419,효창동,5.691176,13792.578502,0.078496,119502.65,0.197999,0.197999
420,후암동,5.042169,26127.273376,0.131738,237745.33,0.369483,0.369483
421,휘경1동,4.811331,18377.557790,0.088421,161263.19,0.249684,0.249684
422,휘경2동,4.066208,18017.144884,0.073261,223366.59,0.296628,0.296628


In [51]:
행정동_면적 = pd.read_csv("./데이터/행정구역(동별)_20231115212741_열린데이터광장_2022.csv")
행정동_면적 = 행정동_면적[["동별(3)", "2022"]].iloc[4:, :].reset_index()
행정동_면적.rename(columns={"동별(3)": "ADM_NM", "2022": "AREA(k㎡)"}, inplace=True)
행정동_면적.drop(columns="index", inplace=True)


In [52]:
행정동별_불투수면_비율 = pd.DataFrame(
    {
        "ADM_NM": 행정동_면적["ADM_NM"],
        "AREA": 행정동_면적["AREA(k㎡)"],
        "도로_건물_면적_km²": 도로_건물["도로_건물_면적_km²"],
        "GREEN_AREA(k㎡)": 행정동별_녹지면적["행정동별_녹지면적(km²)"],
    }
)

In [53]:
행정동별_불투수면_비율.dropna(inplace=True)

In [54]:
행정동별_불투수면_비율

,ADM_NM,AREA,도로_건물_면적_km²,GREEN_AREA(k㎡)
0,사직동,1.23,0.476039,0.039608
1,삼청동,1.49,0.417678,0.052844
2,부암동,2.27,0.479474,0.091714
3,평창동,8.87,0.254072,0.005369
4,무악동,0.36,1.225701,0.004973
...,...,...,...,...
418,가락1동,1.34,0.403071,0.136136
419,가락2동,0.96,0.197999,0.283456
420,문정1동,0.56,0.369483,0.015093
421,문정2동,2.20,0.249684,0.108279


In [55]:
행정동별_불투수면_비율["도로_건물_면적_비율"] = 행정동별_불투수면_비율["도로_건물_면적_km²"].astype(float) / 행정동별_불투수면_비율[
    "AREA"
].astype(float)
행정동별_불투수면_비율["녹지_면적_비율"] = 행정동별_불투수면_비율["GREEN_AREA(k㎡)"].astype(float) / 행정동별_불투수면_비율[
    "AREA"
].astype(float)

In [56]:
행정동별_불투수면_비율

,ADM_NM,AREA,도로_건물_면적_km²,GREEN_AREA(k㎡),도로_건물_면적_비율,녹지_면적_비율
0,사직동,1.23,0.476039,0.039608,0.387024,0.032201
1,삼청동,1.49,0.417678,0.052844,0.280321,0.035466
2,부암동,2.27,0.479474,0.091714,0.211222,0.040403
3,평창동,8.87,0.254072,0.005369,0.028644,0.000605
4,무악동,0.36,1.225701,0.004973,3.404724,0.013814
...,...,...,...,...,...,...
418,가락1동,1.34,0.403071,0.136136,0.300799,0.101594
419,가락2동,0.96,0.197999,0.283456,0.206249,0.295267
420,문정1동,0.56,0.369483,0.015093,0.659792,0.026952
421,문정2동,2.20,0.249684,0.108279,0.113493,0.049218


In [57]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV, KFold, cross_validate
from pca import pca
import seaborn as sb
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import KMeans

In [58]:
MinMaxScaler = MinMaxScaler()
scale_df = MinMaxScaler.fit_transform(df)
sca_df = pd.DataFrame(data=scale_df, columns=df.columns)
pca = PCA()
pca.fit_transform(sca_df)
pca.explained_variance_ratio_

NameError: name 'df' is not defined

In [ ]:
cumsum = np.cumsum(pca.explained_variance_ratio_)
len(cumsum)

In [ ]:
pca_2 = PCA(n_components=6)
pca_2.fit(sca_df)

In [ ]:
plt.figure(figsize=(10, 8))
plt.plot(
    range(1, 18), pca.explained_variance_ratio_.cumsum(), marker="o", linestyle="--"
)
plt.yticks(np.arange(0,1))

plt.ylabel("설명된 분산")
plt.xlabel("차원 수")
plt.rcParams["font.family"] = "Malgun Gothic"
plt.rcParams["font.size"] = 14
plt.rcParams["axes.unicode_minus"] = False

In [ ]:
pca_2_result = pca_2.fit_transform(sca_df)

In [ ]:
results = pd.DataFrame(data=  pca_2.fit_transform(sca_df), columns=['PC-1','PC-2',"PC-3","PC-4","PC-5","PC-6"])

In [ ]:
# 결과 출력
components_df = pd.DataFrame(
    pca_2.components_, columns=sca_df.columns, index=results.columns
)
components_df